# Convert `VRT`  GDAL Virtual Format to `GeoTIFF`

In [1]:
import numpy as np
import json
from google.cloud import storage
from google.oauth2 import service_account
from boto3.session import Session as boto3_session

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from pathlib import Path 
env_path = Path('.') / '.env'

## Utils

In [3]:
def upload_blob_to_GCS(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"
    #private_key = json.loads(os.getenv("PRIVATE_KEY"))
    #credentials = service_account.Credentials(private_key, private_key['client_email'], private_key['token_uri'])
    #storage_client = storage.Client(credentials=credentials, project=private_key['project_id'])
    storage_client = storage.Client.from_service_account_json(os.getenv('PRIVATEKEY_PATH'))
    
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

**upload_blob_to_s3**

In [4]:
def upload_blob_to_S3(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"
    session = boto3_session()
    client = session.client(service_name="s3", aws_access_key_id=os.getenv("S3_ACCESS_KEY_ID"), aws_secret_access_key=os.getenv("S3_SECRET_ACCESS_KEY"))
    client.upload_file(source_file_name, bucket_name, destination_blob_name)

In [ ]:
%%time
#webDAV protocol platform url
base_url = 'https://files.isric.org/projects/tnc/phase2_product2'

GCS_bucket_name = 'vizz-data-transfer'
S3_bucket_name = 'soils-revealed'
file_path = 'SOC_maps/Recent/'

# We create a collection asset in Earth Engine to use as a write target.
#!earthengine create collection projects/soils-revealed/Recent/SOC_stocks
    
for year in np.arange(2004,2019).astype(np.str): #np.arange(2000,2019).astype(np.str):
    print("Creating GeoTIFF from VRT")
    !gdal_translate --config GDAL_SKIP DODS --config GDAL_HTTP_USERPWD {os.getenv("GDAL_HTTP_USERPWD")} -co "BLOCKXSIZE=128" -co "BLOCKYSIZE=128" -co "TILED=YES" -co "BIGTIFF=YES" -co "COMPRESS=DEFLATE" /vsicurl/{base_url}/SOC_{year}_4326.vrt ../data/ISRIC_product2/SOC_{year}_4326.tif
    
    print("Uploading file to GCS bucket")
    upload_blob_to_GCS(GCS_bucket_name, f'../data/ISRIC_product2/SOC_{year}_4326.tif', file_path+f'SOC_{year}_4326.tif')
    
    #print("Uploading file to S3 bucket")
    #upload_blob_to_S3(S3_bucket_name, f'../data/ISRIC_product2/SOC_{year}_4326.tif', file_path+f'SOC_{year}_4326.tif')
    
    print("Uploading file to GEE")
    !earthengine upload image --time_start {year}-01-01 --time_end {year}-12-31 --asset_id projects/soils-revealed/Recent/SOC_stocks/SOC_stocks_{year} gs://vizz-data-transfer/SOC_maps/Recent/SOC_{year}_4326.tif  
    
    print("Removing GeoTIFF file")
    os.remove(f'../data/ISRIC_product2/SOC_{year}_4326.tif')

Creating GeoTIFF from VRT
Input file size is 158580, 63599
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading file to GCS bucket
File ../data/ISRIC_product2/SOC_2004_4326.tif uploaded to SOC_maps/Recent/SOC_2004_4326.tif.
Uploading file to GEE
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

W0720 14:31:33.809015 140736002478976 __init__.py:46] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/Users/ikersanchez/anaconda3/envs/geoenv/lib/python3.8/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ikersanchez/anaconda3/envs/geoenv/lib/python3.8/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, 